# Imports

In [4]:
import pandas as pd
import requests
import numpy as np
import json
from bs4 import BeautifulSoup as bs
import random
import time
import json
import pickle

# Scraper

Toma los usuarios de MAL y los guarda en un archivo pickle. Cada diccionario creado es de la forma:

```python
formato = {
    id_usuario: {nombre: "name", tipo: "tipo", 1: [rating, anime_id], 2: [rating, anime_id], ...},
    id_usuario: {nombre: "name", tipo: "tipo", 1: [rating, anime_id], 2: [rating, anime_id], ...}
}
```

Cada review es una lista [rate, estado]. Y el estado puede ser:

1. watching
2. completed
3. on hold
4. dropped,
5. desconocido,
6. plan to watch.

In [5]:
### Headers para la animelist.
headers = {
    "accept": "application/json",
    "X-MAL-CLIENT-ID": "dba80244ba88ffa1c4914806e9ccb3e4"
}

### Parámetros para obtener la animelist del usuario.
animelist_params = {
    "fields": "list_status",
    "sort": "list_score",
    "limit": "1000"
}

In [8]:
def extraer_listas(usuarios: dict, listas: dict, head, parametros):
    """
    Usuarios debe ser un diccionario con las (user_id : user_name), listas son las listas previamente recolectadas, en caso de que no hayan debe ser
    un diccionario vacio, retorna las listas de usuarios faltantes en "listas"

    Ambos diccionarios deben estar respecto a keys numericas, debido a eso usuarios debe convertir sus keys a int antes de ingresarse como argumento.

    Es recomendado guardar el output en pickles para ser compacto en el espacio.
    """
    listas_usuarios = {}
    for user_id in (usuarios.keys() - listas.keys()):
        user_id = int(user_id)
        user_name = usuarios[user_id]
        time.sleep(3)
        # Inicializar la lista con el nombre de usuario
        listas_usuarios[user_id] = {"nombre": user_name}

        # Tratamos de obtener la lista del usuario las veces que sea necesaria.
        # Si por alguna razon es privada o vacia, se rompe el ciclo.
        while True:
            try:
                # Obtener lista
                user_animelist = requests.get(
                    url="https://api.myanimelist.net/v2/users/"+user_name+"/animelist",
                    headers=head,
                    params=parametros
                )

                # Manejar errores / Status codes.
                if user_animelist.status_code == 429: # Demasiadas peticiones
                    print("Demasiadas peticiones")
                    time.sleep(120)
                    continue
                elif user_animelist.status_code == 403: # La Animelist del usuario es privada.
                    listas_usuarios[user_id]["tipo"] = "privada"
                    break
                elif user_animelist.status_code == 404: # No se encontro el usuario.
                    listas_usuarios[user_id]["tipo"] = "perdido"
                    break
                elif (user_animelist.status_code // 100) == 5: # Error en el servidor.
                    print(user_animelist.status_code)
                    print(user_animelist.json())
                    print(user_id, user_name)
                    time.sleep(120)
                    continue
                elif user_animelist.status_code != 200: # Error desconocido
                    print(user_animelist.status_code)
                    print(user_animelist.json())
                    print(user_id, user_name)
                    time.sleep(120)
                    continue

                # Obteniendo la lista en si.
                user_json = user_animelist.json()["data"]
                if not user_json: # La Animelist es vacia.
                    listas_usuarios[user_id]["tipo"] = "vacia"
                    break
                listas_usuarios[user_id]["tipo"] = "publica"
                # Transformador categorico:
                transformar = {
                    "watching": 1,
                    "completed": 2,
                    "on_hold": 3,
                    "dropped": 4,
                    "desconocido": 5,
                    "plan_to_watch": 6
                }

                # Añadiendo la informacion a la lista del usuario
                for anime in user_json:
                    anime_id = int(anime["node"]["id"])
                    anime_rate = int(anime["list_status"]["score"])
                    anime_status = anime["list_status"].get("status", "desconocido")
                    int_status = transformar.get(anime_status, 5)
                    listas_usuarios[user_id][anime_id] = [anime_rate, int_status]
                break # Extraccion exitosa.
            except KeyboardInterrupt: # Usuario saltado por algun error etc.
                print("Usuario saltado")
                print(user_animelist.status_code)
                print(user_animelist.json())
                print(user_id, user_name)
                break
    # Estas listas idealmente deben ser guardadas en pickle para usar eficientemente el espacio.
    return listas_usuarios

# Extracción previa

In [61]:
extraer_listas("usuarios_1.json", "listas_usuarios_1.pickle", headers, animelist_params)

In [5]:
for i in range(7, 11):
    extraer_listas(f"usuarios_{i}.json", f"listas_usuarios_{i}.pickle", headers, animelist_params)

504
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
404
404
500
500
500
500
500
500
500
500
500
500
500
500
500
500
504
500
500
500
500
500
500
500
500
500
500
500
500
500
500
504
504
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
504
504
500
500
500
500
500
500
500
500
500
500
500
500
500
504
504
500
500
500
500
500
500
500
500
500
500
500
504
504
500
500
500
500
500
500
500
500
500
500
500
500
500
504
504
500
500
500
500
500
504
504
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
504
504
500
500
500
500
500
500
504
500
500
500
500
500
500
500
500
500
500
500
500
500
500
504
504
500
500
500
500
500
500
500
500
500
504
504
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
504
500
500
500
500
500
500
500
500
500
500
500
500
504
500
500
500
500
500
500
500
500
500
500
500
500
500
500
504
500
500
500
500
500
500
500
500
500
500
500
500
500
504
504
500
500
500
500


In [6]:
for i in range(11, 21):
    extraer_listas(f"usuarios_{i}.json", f"listas_usuarios_{i}.pickle", headers, animelist_params)

504
504
504
504
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
404
404
404


KeyError: 'status'

In [11]:
for i in range(16, 21):
    extraer_listas(f"usuarios_{i}.json", f"listas_usuarios_{i}.pickle", headers, animelist_params)

{'node': {'id': 42897, 'title': 'Horimiya', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/1695/111486.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/1695/111486l.jpg'}}, 'list_status': {'status': 'completed', 'score': 9, 'num_episodes_watched': 13, 'is_rewatching': False, 'updated_at': '2021-07-30T15:02:25+00:00'}}
{'node': {'id': 121, 'title': 'Fullmetal Alchemist', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/10/75815.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/10/75815l.jpg'}}, 'list_status': {'status': 'completed', 'score': 0, 'num_episodes_watched': 51, 'is_rewatching': False, 'updated_at': '2021-07-29T14:42:06+00:00'}}
{'node': {'id': 199, 'title': 'Sen to Chihiro no Kamikakushi', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/6/79597.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/6/79597l.jpg'}}, 'list_status': {'status': 'completed', 'score': 0, 'num_episodes_watched': 1, 'is_r

KeyError: 'status'

# Completar datos perdidos.

In [4]:
with open("usuarios_1.json", "r", encoding="utf-8") as archivo:
    usuarios = {int(x[0]) : x[1] for x in json.load(archivo).items()}
with open("listas_usuarios_1.pickle", "rb") as archivo:
    listas = pickle.load(archivo)
    for user in listas:
        listas[user]["tipo"] = "publica"
datos_faltantes = extraer_listas(usuarios, listas, headers, animelist_params)
datos_faltantes

{487429: {'nombre': 'fallenangel1007', 'tipo': 'vacia'},
 468999: {'nombre': 'Kirby101ify', 'tipo': 'vacia'},
 415768: {'nombre': 'erwin99991', 'tipo': 'vacia'},
 653347: {'nombre': 'clarkroberts1129', 'tipo': 'vacia'},
 616483: {'nombre': 'noelsheprd24', 'tipo': 'vacia'},
 458787: {'nombre': 'ListofShame939', 'tipo': 'vacia'},
 380969: {'nombre': 'Zackry896', 'tipo': 'vacia'},
 415786: {'nombre': 'obsess_sadhbh', 'tipo': 'vacia'},
 720941: {'nombre': 'lancekostas63', 'tipo': 'vacia'},
 67633: {'nombre': 'latentimage', 'tipo': 'vacia'},
 696373: {'nombre': 'landonsantos511', 'tipo': 'vacia'},
 645173: {'nombre': 'adolphsimmon819', 'tipo': 'vacia'},
 708671: {'nombre': 'kenttyson1127', 'tipo': 'vacia'},
 30794: {'nombre': 'Climhazzard', 'tipo': 'vacia'},
 159820: {'nombre': 'kirkuk_lady', 'tipo': 'vacia'},
 143448: {'nombre': 'Swerh', 'tipo': 'vacia'},
 419928: {'nombre': 'phen02', 'tipo': 'vacia'},
 772193: {'nombre': 'manuelharmon23', 'tipo': 'vacia'},
 577633: {'nombre': 'lestermclau

In [5]:
union = (listas | datos_faltantes)
union

{277950: {'nombre': 'mxz1987', 1559: [10, 1], 'tipo': 'publica'},
 39110: {'nombre': 'xxbladexx20',
  24: [10, 2],
  519: [10, 2],
  846: [10, 2],
  849: [10, 2],
  1530: [10, 2],
  1887: [10, 2],
  2924: [10, 2],
  4163: [10, 1],
  4181: [10, 1],
  193: [9, 2],
  355: [9, 2],
  590: [9, 2],
  853: [9, 2],
  856: [9, 2],
  1195: [9, 2],
  1568: [9, 2],
  1722: [9, 2],
  2026: [9, 2],
  2104: [9, 2],
  2129: [9, 2],
  2167: [9, 2],
  2615: [9, 2],
  2926: [9, 2],
  2993: [9, 2],
  3011: [9, 2],
  3456: [9, 2],
  3467: [9, 1],
  3577: [9, 1],
  4535: [9, 1],
  125: [8, 2],
  260: [8, 2],
  261: [8, 2],
  469: [8, 2],
  1555: [8, 2],
  1569: [8, 2],
  1692: [8, 2],
  2476: [8, 2],
  2986: [8, 2],
  3230: [8, 2],
  3298: [8, 2],
  3299: [8, 2],
  3455: [8, 1],
  3627: [8, 2],
  4789: [8, 2],
  836: [7, 2],
  910: [7, 2],
  1691: [7, 2],
  1727: [7, 4],
  1815: [7, 2],
  1840: [7, 2],
  1965: [7, 3],
  3759: [7, 2],
  1218: [6, 2],
  2787: [6, 1],
  3712: [6, 1],
  121: [5, 4],
  863: [5, 4

In [6]:
with open(f"listas_usuarios_1.pickle", "wb") as archivo:
    pickle.dump(union, archivo)

In [7]:
for i in range(8, 16):
    with open(f"usuarios_{i}.json", "r", encoding="utf-8") as archivo:
        usuarios = {int(x[0]) : x[1] for x in json.load(archivo).items()}
    with open(f"listas_usuarios_{i}.pickle", "rb") as archivo:
        listas = pickle.load(archivo)
        for user in listas:
            listas[user]["tipo"] = "publica"
    datos_faltantes = extraer_listas(usuarios, listas, headers, animelist_params)
    union = (listas | datos_faltantes)
    with open(f"listas_usuarios_{i}.pickle", "wb") as archivo:
        pickle.dump(union, archivo)

{'node': {'id': 120, 'title': 'Fruits Basket', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/4/75204.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/4/75204l.jpg'}}, 'list_status': {'status': 'watching', 'score': 9, 'num_episodes_watched': 0, 'is_rewatching': False, 'updated_at': '2016-11-07T20:27:57+00:00'}}
{'node': {'id': 1, 'title': 'Cowboy Bebop', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/4/19644.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/4/19644l.jpg'}}, 'list_status': {'status': 'on_hold', 'score': 0, 'num_episodes_watched': 0, 'is_rewatching': False, 'updated_at': '2019-12-27T15:32:30+00:00'}}
{'node': {'id': 199, 'title': 'Sen to Chihiro no Kamikakushi', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/6/79597.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/6/79597l.jpg'}}, 'list_status': {'status': 'completed', 'score': 0, 'num_episodes_watched': 1, 'is_rewatching': False, 'u

KeyboardInterrupt: 